In [2]:
import requests
import pandas as pd
import glob

In [8]:
# récupérer tous les fichiers du dossier videos et dédupliquer les lignes
files = glob.glob("videos/*.csv")

df = []
for f in files:
    csv = pd.read_csv(f)
    df.append(csv)
df = pd.concat(df,ignore_index=True)
df.drop_duplicates(keep="first", inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998 entries, 0 to 1997
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   video_id      1998 non-null   object
 1   title         1998 non-null   object
 2   description   1502 non-null   object
 3   channel       1998 non-null   object
 4   published_at  1998 non-null   object
 5   duration      1996 non-null   object
 6   views         1998 non-null   int64 
 7   likes         1998 non-null   int64 
 8   comments      1998 non-null   int64 
 9   channel_id    1998 non-null   object
 10  category_id   1998 non-null   int64 
 11  language      1992 non-null   object
dtypes: int64(4), object(8)
memory usage: 187.4+ KB


In [ ]:
# transformer la colonne duration en sec

In [ ]:
# créer la colonne taux d'engagement

In [ ]:
# colonnes avec valeurs nulls
df.isna().sum()

video_id          0
title             0
description     496
channel           0
published_at      0
duration          2
views             0
likes             0
comments          0
channel_id        0
category_id       0
language          6
dtype: int64

In [ ]:
# suppression : certaines langues ? 